# Preprocessing for Deep Learning NLP   

## 1. Introduction
在深度学习应用于自然语言处理（NLP）的背景下，数据预处理是一个至关重要的步骤，它通常包括以下几个环节：

1. **文本清洗（Text cleaning）**：
   - **去除噪声**：例如去除HTML标签、特殊字符等。
   - **规范化文本**：比如将所有文本转换为小写，以减少大小写造成的变体。
   - **分词（Tokenization）**：这是NLP中的一个基本步骤，涉及将连续的文本字符串分解为单独的单词或符号。
   - **去除停用词（Stop word removal）**：诸如“the”、“is”、“in”等单词通常对于理解文本的含义不是很有用，可以被移除。
   - **词干提取（Stemming）/词形还原（Lemmatization）**：比如将“running”还原为“run”，有助于模型理解不同词形的共同含义。

2. **构建词汇表（Vocabulary construction）**：
   - **确定模型的词汇**：基于训练数据创建一个单词集合。

3. **词嵌入（Word embeddings）**：
   - **单词向量化**：使用预训练的词向量（如GloVe、Word2Vec）或在训练过程中学习词向量。

4. **序列填充（Padding）/截断（Truncation）**：
   - **统一序列长度**：由于神经网络需要固定长度的输入，过长的序列可能需要截断，而过短的序列则需要通过填充额外的零或特定的占位符来扩展。

5. **构建标签（Label construction）**：
   - **准备监督学习标签**：对于有监督学习任务，比如分类或序列标注，需要准备相应的标签。

6. **数据增强（Data augmentation）**：
   - **扩展训练数据**：使用各种技术（如同义词替换、随机删除、句子重组）来创建更多的训练样本，有助于提高模型的鲁棒性和泛化能力。

7. **分批处理（Batching）**：
   - **准备用于训练的数据批次**：组织数据以便以批次的形式进行有效训练。

根据特定的任务和模型架构，预处理的具体步骤和实现可能会有所不同。例如，对于Transformer模型，通常会添加特殊的起始（[CLS]）和分隔（[SEP]）标记。而对于句子对任务，则可能需要确保两个句子合并后的长度不超过模型的最大序列长度。

In [1]:
import spacy
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from collections import Counter
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import GloVe
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab
from torch.nn import Embedding
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

## 2. Text Cleaning
spaCy是一个流行的自然语言处理库，提供了许多文本处理功能，包括分词、词形还原、命名实体识别等。在本节中，我们将使用spaCy来清理文本数据。

spaCy提供多种不同类型和大小的预训练模型，适用于多种语言。这些模型包括：

1. **核心模型（Core Models）**：
   - 包含了词性标注、句法分析、命名实体识别等功能。
   - 对于英语，提供了不同大小的模型，如`en_core_web_sm`（小型）、`en_core_web_md`（中型）、`en_core_web_lg`（大型），还有`en_core_web_trf`（基于transformer的模型）。

2. **多语种和语言特定模型**：
   - spaCy提供了对多种语言的支持，如西班牙语、法语、德语、俄语等。
   - 这些模型有不同的大小版本，一般也遵循`xx_core_web_sm`、`xx_core_web_md`、`xx_core_web_lg`的命名规范（`xx`是语言代码）。

3. **Transformer模型**：
   - 如`en_core_web_trf`，这类模型基于预训练的Transformer模型，如BERT或RoBERTa，它们通常提供更优的性能，尤其在理解上下文和复杂语义方面。

除了核心模型，spaCy社区还贡献了额外的模型和管道，这些可以通过spaCy官方网站和spaCy Universe来发现。这些额外的资源可能包括特定于某个领域的模型或用于特定任务的工具，比如情感分析、文本分类等。

模型的选择依赖于具体的任务和所需语言。一般而言，小型模型下载快、占用空间少，适合快速原型设计或当环境资源有限时使用；大型模型和基于Transformer的模型提供更高的精度和更好的性能，但下载和加载时间更长，需要更多的资源。

可以使用spaCy的命令行工具或在Python代码中使用`spacy.cli.download`函数来下载这些模型。使用时，需要确保选择的模型与你使用的spaCy版本兼容。

In [2]:
# 加载英语模型
nlp = spacy.load("en_core_web_sm")

# 处理文本
text = "Apple is looking at buying U.K. startup for $1 billion. The foxes walk into the box. The dog was barking."
doc = nlp(text)

# 打印文本
print(doc.text)

# 遍历文档中的句子
print('\nSentences:')
for sent in doc.sents:
    print(sent.text)
    
# 遍历文档中的命名实体
print('\nNamed Entities:')
for ent in doc.ents:
    print(ent.text, ent.label_)

# 打印每个token的文本和词性标注
print('\nTokens and their POS tags:')
for token in doc:
    print(token.text, token.pos_)

# 名词短语
print('\nNoun chunks:')
for chunk in doc.noun_chunks:
    print(chunk.text)
    
# 词形还原
print('\nLemmatized words:')
for token in doc:
    print(token.text, token.lemma_)


Apple is looking at buying U.K. startup for $1 billion. The foxes walk into the box. The dog was barking.

Sentences:
Apple is looking at buying U.K. startup for $1 billion.
The foxes walk into the box.
The dog was barking.

Named Entities:
Apple ORG
U.K. GPE
$1 billion MONEY

Tokens and their POS tags:
Apple PROPN
is AUX
looking VERB
at ADP
buying VERB
U.K. PROPN
startup NOUN
for ADP
$ SYM
1 NUM
billion NUM
. PUNCT
The DET
foxes NOUN
walk VERB
into ADP
the DET
box NOUN
. PUNCT
The DET
dog NOUN
was AUX
barking VERB
. PUNCT

Noun chunks:
Apple
U.K.
The foxes
the box
The dog

Lemmatized words:
Apple Apple
is be
looking look
at at
buying buy
U.K. U.K.
startup startup
for for
$ $
1 1
billion billion
. .
The the
foxes fox
walk walk
into into
the the
box box
. .
The the
dog dog
was be
barking bark
. .


In [3]:
# 加载本地的imdb数据集，arrow格式
data_files = {
    "train": './data_cache/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/imdb-train.arrow',
    "test": "./data_cache/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/imdb-test.arrow",
}
dataset = load_dataset('arrow', data_files=data_files)

# split the dataset,only use 1000 samples for demo
train_dataset = dataset['train'].select(range(1000)) # select 函数用于选择数据集的子集，这里选择前1000个样本
test_dataset = dataset['test'].select(range(500))

# 获取文本和标签
train_text = [sample["text"] for sample in train_dataset]
train_label = [sample["label"] for sample in train_dataset]
test_text = [sample["text"] for sample in test_dataset]
test_label =  [sample["label"] for sample in test_dataset]

### 2.1 spaCy文本预处理
一般来说，spaCy的文本预处理包括以下几个步骤：
- 加载spaCy模型
- 定义文本预处理函数：
    - 使用`nlp.pipe`方法高效地处理大量文本,禁用不需要的组件以提高效率。
    - 去除停用词、标点和空格，并进行词形还原或其他处理。
    - 将处理后的文本重新组合成字符串。
- 对train_text和test_text进行预处理。

In [4]:
# 加载Spacy模型
nlp = spacy.load("en_core_web_sm")

# 定义文本预处理函数
# 该函数使用Spacy进行文本预处理，包括分词、词形还原和去除停用词
# 使用Spacy的管道（pipe）机制，可以高效地处理大量文本，优于直接使用nlp(text)方法；可选择禁用不需要的组件以提高效率
def preprocess_text(texts):
    cleaned_texts = []
    for doc in nlp.pipe(texts, disable=["parser", "ner"]):
        # 去除停用词、标点和空格，并进行词形还原,转换为小写
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        cleaned_text = " ".join(tokens)
        cleaned_texts.append(cleaned_text)
    return cleaned_texts

# 对训练集和测试集进行预处理
train_text_clean = preprocess_text(train_text)
test_text_clean = preprocess_text(test_text)

# 预处理后的文本可以用于后续的NLP任务，比如特征提取、模型训练等

In [72]:
# 打印预处理后的文本
print(train_text_clean[0])

rent curious yellow video store controversy surround release 1967 hear seize u.s. custom try enter country fan film consider controversial myself.<br /><br />the plot center young swedish drama student name lena want learn life particular want focus attention make sort documentary average swede think certain political issue vietnam war race issue united states ask politician ordinary denizen stockholm opinion politic sex drama teacher classmate marry men.<br /><br />what kill curious yellow 40 year ago consider pornographic sex nudity scene far shoot like cheaply porno countryman mind find shocking reality sex nudity major staple swedish cinema ingmar bergman arguably answer good old boy john ford sex scene films.<br /><br />i commend filmmaker fact sex show film show artistic purpose shock people money show pornographic theater america curious yellow good film want study meat potato pun intend swedish cinema film plot


## 3.Vecorization
文本向量化是将文本数据转换为数值形式的过程，以便计算机能够理解和处理。在自然语言处理中，文本向量化通常包括以下几种方法：
1. 稀疏表达：TDM（Term-Document Matrix）
    - Bag of Words（词袋模型）：将文本表示为词汇表中单词的出现次数。
    - TF-IDF（Term Frequency-Inverse Document Frequency）：将文本表示为单词的TF-IDF值，以衡量单词在文档中的重要性。
2. 密集表达：Word Embeddings
    - Word2Vec：将单词映射到低维空间的向量表示，以捕获单词之间的语义关系。
    - GloVe（Global Vectors for Word Representation）：基于全局词频统计的词向量。
    - FastText：Facebook提出的一种基于字符级n-gram的词向量方法。
    - BERT（Bidirectional Encoder Representations from Transformers）：基于Transformer的预训练模型，提供了上下文相关的词向量。

对于深度学习模型，通常使用词嵌入（Word Embeddings）来表示文本数据。在PyTorch中，可以使用`torch.nn.Embedding`层来加载预训练的词向量，或在训练过程中学习词向量。

#### Word2Vec词向量
Word2Vec是一种常用的词向量表示方法，它通过训练神经网络模型来学习单词的分布式表示。Word2Vec模型通常有两种架构：
- **Skip-gram**：通过给定中心词预测上下文词。
- **CBOW（Continuous Bag of Words）**：通过给定上下文词预测中心词。

参数说明：
- `tokenized_texts`：分词后的文本，作为训练数据输入。
- `sg=1`：使用Skip-Gram模型。如果设置为`0`，则使用CBOW模型。
- `min_count=1`：词语出现的最小次数。此参数确保了只有至少出现一次的词语才会被纳入训练。
- `window=3`：当前词与预测词在一个句子中的最大距离。
- `vector_size=100`：特征向量的维度大小。

Word2Vec模型的训练过程通常使用负采样（Negative Sampling）或层次Softmax（Hierarchical Softmax）来提高训练效率。Word2Vec模型的输出是每个单词的词向量，可以用于后续的文本分类、聚类、相似度计算等任务。

输出的数据格式为：[num_words, embedding_dim]，其中`num_words`是词汇表中单词的数量，`embedding_dim`是词向量的维度。

In [17]:
# word2vec词向量
from gensim.models import Word2Vec

# 定义Word2Vec模型，使用tensor储存词向量
def text_to_word2vec(texts):
    # vector_size表示词向量的维度，window表示上下文窗口大小，min_count表示最小词频，workers表示线程数
    w2v_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=-1, sg=1)
    word_vectors = w2v_model.wv
    return word_vectors

train_word2vec = text_to_word2vec(train_text_clean)
test_word2vec = text_to_word2vec(test_text_clean)

# 使用tensor储存词向量
train_word2vec_tensor = torch.tensor([train_word2vec[token] for token in train_word2vec.key_to_index]) # [85, 100]: 85个词，每个词的词向量维度为100
test_word2vec_tensor = torch.tensor([test_word2vec[token] for token in test_word2vec.key_to_index]) # [77,100]: 77个词，每个词的词向量维度为100

# 打印词嵌入
print(train_word2vec_tensor)

tensor([[-5.3623e-04,  2.3643e-04,  5.1033e-03,  ..., -7.0416e-03,
          9.0146e-04,  6.3925e-03],
        [-8.6197e-03,  3.6657e-03,  5.1899e-03,  ..., -2.3915e-03,
         -9.5101e-03,  4.5059e-03],
        [ 9.4564e-05,  3.0773e-03, -6.8126e-03,  ...,  5.1259e-04,
          8.2131e-03, -7.0190e-03],
        ...,
        [ 5.0281e-03, -3.6851e-03,  3.4227e-03,  ...,  3.0911e-04,
          7.6837e-03,  1.7421e-03],
        [-9.0065e-03,  5.3365e-03,  3.7596e-03,  ...,  7.5535e-03,
          3.9018e-03,  7.7782e-03],
        [ 4.2228e-03,  1.7857e-04,  4.7847e-03,  ...,  2.8274e-03,
         -1.6949e-03,  8.0678e-04]])


#### GloVe词向量
GloVe是一种基于全局词频统计的词向量方法，它通过对词共现矩阵进行奇异值分解（SVD）来学习词向量。GloVe词向量通常在大型语料库上进行预训练，可以直接使用预训练的GloVe词向量，也可以在训练过程中学习词向量。

输出的数据格式为：[num_samples, embedding_dim]，其中`num_samples`是样本数量,即原数据的文档数量，`embedding_dim`是词向量的维度。

In [16]:
# GloVe词向量，6B表示使用6亿词的预训练词向量，dim表示词向量的维度
glove = GloVe(name='6B', dim=100)

# 定义词嵌入函数，将文本转换为词向量
def text_to_embedding(texts, glove):
    embeddings = []
    for text in texts:
        tokens = text.split() # 分词
        embedding = [glove[token] for token in tokens if token in glove.stoi] # 获取词向量
        
        if len(embedding) > 0:
            embedding = torch.stack(embedding).mean(0)  # 取平均得到整个句子的嵌入表示
        else:
            # 如果句子中的所有词都不在词向量中，则使用全零向量表示
            embedding = torch.zeros(glove.vectors.shape[1])
        embeddings.append(embedding)
    return torch.stack(embeddings)

train_embeddings = text_to_embedding(train_text_clean, glove) # [1000, 100]: 1000个样本，每个样本的词向量维度为100
test_embeddings = text_to_embedding(test_text_clean, glove) # [500, 100]: 500个样本，每个样本的词向量维度为100

# 打印词嵌入
print(train_embeddings)

tensor([[ 0.1028,  0.1800,  0.2473,  ..., -0.1776,  0.2904,  0.1504],
        [ 0.0249,  0.2207,  0.2789,  ..., -0.3625,  0.2532,  0.0806],
        [-0.1845,  0.2870,  0.3115,  ..., -0.2843,  0.2605,  0.2389],
        ...,
        [ 0.0226,  0.1057,  0.4098,  ..., -0.1401,  0.3588,  0.1761],
        [ 0.0239,  0.1033,  0.2309,  ..., -0.1844,  0.2863,  0.0968],
        [-0.0267,  0.1591,  0.2689,  ..., -0.1677,  0.2733,  0.0952]])


## nn.Embedding
在PyTorch中，可以使用`torch.nn.Embedding`层来加载预训练的词向量，或在训练过程中学习词向量。`torch.nn.Embedding`层的输入是一个整数张量，表示单词的索引，输出是对应的词向量。

可以将文本数据直接转换为整数序列，并将这些整数序列作为模型的输入，而无需手动进行词嵌入。

1. 分词和预处理：对原始训练集进行分词和预处理，包括去除标点符号、停用词等，以及将文本转换为小写形式。

2. 构建词汇表：根据预处理后的训练集构建词汇表，将每个单词映射到一个唯一的整数。

3. 将文本转换为整数序列：将预处理后的文本数据转换为整数序列，其中每个单词都用其在词汇表中的整数表示。可以使用Python的字典或者torchtext库等工具来实现这一步骤。

4. 准备数据集：将转换后的整数序列组织成批次，并根据需要填充或截断成相同长度，以便输入到模型中。

5. 训练模型：将整数序列的批次作为模型的输入，直接进行训练。

通过这种方式，您可以直接将原始文本数据转换为整数序列，并将其传递给模型进行训练和测试，无需手动进行词嵌入。这种方法简化了数据准备过程，并使模型的使用更加方便。

In [47]:
from torch.nn.utils.rnn import pad_sequence
from torch import LongTensor

# 定义填充函数
def pad_sequences(sequences, padding_value=0):
    return pad_sequence([LongTensor(seq) for seq in sequences], batch_first=True, padding_value=padding_value)

# 修改你的函数，添加填充步骤
def text_to_int_sequence(texts, vocab):
    int_sequences = []
    for text in texts:
        int_sequence = [vocab[word] for word in text.split() if word in vocab]
        int_sequences.append(torch.LongTensor(int_sequence))  # 转换为长整型
    return pad_sequences(int_sequences)

# 重新转换文本为整数序列，并进行填充
train_int_sequences = text_to_int_sequence(train_text_clean, vocab)
test_int_sequences = text_to_int_sequence(test_text_clean, vocab)

# 打印转换和填充后的整数序列示例
print(train_int_sequences[0])
print(test_int_sequences[0])

tensor([ 222, 1061, 1845,  176,  723, 5177, 1200,  280, 7396,  184, 4060, 2003,
        2815,   33,  951,  604,   88,    3,  262, 1846, 7397,    1,   24,   17,
        2179,   93, 2816,  394,  370,  285, 1847,   30,  481,   46,  605,   30,
         620,  496,   35,  164,  952,  584, 5178,   12,  903, 1012,  518, 2180,
         332,  861,  518, 3339, 3340,  260, 2817, 2181, 7398, 4061,  621, 1695,
         142,  394,  585, 2818, 1013, 7399,    1,  827,   43, 1061, 1845,  724,
          41,  413,  262, 4062,  142,  467,   11,   85,  152,    4, 3341, 2467,
        7400,  158,   28, 1696,  468,  142,  467,  456, 5179, 2816,  265, 7401,
        3342, 4063, 1014,    6,   53,  165,  348, 2468,  142,   11, 5180,    1,
          69, 5181,  414,   76,  142,   79,    3,   79, 2004,  568, 1201,   22,
         103,   79, 4062,  371,  674, 1061, 1845,    6,    3,   30, 1697, 2819,
        4064, 2469, 1113, 2816,  265,    3,   17,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   